In [9]:
# External libraries
import numpy as np
import time
import matplotlib.pyplot as plt
import matplotlib.style as style
import pandas as pd
import astropy
import scipy
from filterpy.kalman import KalmanFilter 
from filterpy.common import Q_discrete_white_noise
from scipy.linalg import block_diag
from astropy import units as u
from poliastro.bodies import Earth, Mars, Sun, Moon
from poliastro.twobody import Orbit
from poliastro.plotting import OrbitPlotter2D
from poliastro.plotting import OrbitPlotter3D
from sklearn import linear_model, datasets
import glob
# Own Libraries
from utility.utils import *
from KalmanFilter.kf import *
from Detect.detector import *
from Match.pair import *
from Match.icp import *

style.use('seaborn-paper')
%matplotlib tk

global km2px, deg2km, px2km, deg2px

def find_pos(idx):
    # Loading All Images:
    dict = load_all_images(dt=10)
    # Img:
    filename = dict[str(idx+1)]
    img=cv2.imread(filename)
    # Detection:
    t1 = time.time()
    craters_det = detect(img)
    # Removing minor craters:
    craters_det = craters_det[craters_det[:,2] > 15]
    t2 = time.time()
    print(f'Detection Time:{t2-t1:.2f}\n')
    # Pandas DataFrame:
    df_craters_det = sort_mat(craters_det)
    # Find all triplets:
    t1 = time.time()
    triplets = find_all_triplets(craters_det)
    triplets_det= pd.DataFrame(triplets, columns=['Angle1','Angle2','Angle3','des1','des2','des3','x1','y1','r1','x2','y2','r2','x3',   'y3','r3'])
    triplets_det.shape
    t2 = time.time()
    print('\n')
    print(f'Total craters founded:{craters_det.shape[0]}')
    print(f'Number of total combinations:{triplets_det.shape[0]}\nComputational time: {t2-t1:.2f} s')

###############################################################################################################
    # Opening Database:
    # DB = pd.read_csv('DATA/H_L_combined.csv')
    DB = pd.read_csv('DATA/lunar_crater_database_robbins_2018.csv')

    # Filtering DATABASE:
    MULTIPLAYER = 1.
    span = 3.29/2   *MULTIPLAYER # TODO: Aggiustare l'immagine di catalogo a grandezza effettiva!
    lat_bounds=[-span, span]
    get_lon = float(filename.split('_')[-1].split('jpg')[0][:-2])
    lon_bounds=[get_lon-span,get_lon+span]
    # craters_cat = CatalogSearch(DB, lat_bounds, lon_bounds, CAT_NAME='COMBINED')
    craters_cat = CatalogSearch(DB, lat_bounds, lon_bounds, CAT_NAME='ROBBINS')

    # FIND TRIPLETS:
    if craters_cat is not None:
        km2deg = 1/deg2km
        craters_cat = craters_cat[(craters_cat.Diam < 35)&(craters_cat.Diam > 4)]
        craters_cat['Diam']*=0.5*km2deg # km --- > deg

        craters_cat_m = np.array(craters_cat)
        t1 = time.time()
        triplets_cat_m = find_all_triplets(craters_cat_m)
        triplets_cat = pd.DataFrame(triplets_cat_m, columns=['Angle1','Angle2','Angle3','des1','des2','des3','lon1','lat1', 'r1','lon2',    'lat2','r2','lon3','lat3','r3'])
        triplets_cat['r1'] *= deg2km
        triplets_cat['r2'] *= deg2km
        triplets_cat['r3'] *= deg2km
        t2 = time.time()
        print(f'Total craters catalogued:{craters_cat.shape[0]+1}')
        print(f'Number of total combinations:{triplets_cat.shape[0]}\nComputational time: {t2-t1:.2f} s')
    else:
        print('No craters in cat!')
###############################################################################################################
    tol1 = 7.

    t1 = time.time()
    QUERY1 = triplets_cat
    QUERY2 = triplets_det
    QUERY1 = dropduplicates(QUERY1)
    QUERY2 = dropduplicates(QUERY2) 

    if QUERY1.shape[0]<QUERY2.shape[0]:
        mode = 'natural'
        joins, items = inner_join(QUERY1, QUERY2, tol1)
    else:
        mode = 'inverse'
        joins, items = inner_join(QUERY2, QUERY1, tol1)
    print(f'Mode:{ mode}')
    t2 = time.time()
    print(f'Computational time: {t2-t1:.2f} s\nPossible list Combinations: {len(items)}')
###############################################################################################################
    t1 = time.time()
    tol2 = 1.
    S, iss = [], []
    for i in range(len(joins)):
        join = joins[i]
        des1, des2, des3 = items[i].des1, items[i].des2, items[i].des3
        s=join[ (abs(join.des1 - des1) < tol2) & (abs(join.des2 - des2) < tol2) & (abs(join.des3 - des3) < tol2)\
              | (abs(join.des1 - des2) < tol2) & (abs(join.des2 - des3) < tol2) & (abs(join.des3 - des1) < tol2)\
              | (abs(join.des1 - des3) < tol2) & (abs(join.des2 - des1) < tol2) & (abs(join.des3 - des2) < tol2)]

        if s.shape[0] > 0:
            S.append(s)
            iss.append(items[i])
    t2 = time.time()
    print(f'Computational time: {t2-t1:.2f} s\nPossible list Combinations: {len(S)}')
###############################################################################################################
    # FIND TRUE COMBINATIONS:
    CAMx, CAMy = ( (lon_bounds[0] + lon_bounds[1]) / 2, (lat_bounds[0] + lat_bounds[1]) / 2) # Location Absolute

    Is, Js = [], []
    for I in range(len(iss)):
        row1 = iss[I]
        for J in range(S[I].shape[0]):
            if check_sol(I,J, 0.1, mode, S, iss) & check_sol2(I,J, 0.14, mode, S, iss, CAMx, CAMy): # TODO: Aggiungere checksol3:   confronto crts
                    Is.append(I)
                    Js.append(J)
                    print(I,J)
    Is = np.array(Is)
    Js = np.array(Js)
###############################################################################################################
    if len(Js)!=0:    

        H, px2deg = H_estimation(Is, Js, mode, S, iss, CAMx, CAMy)
        print(f'Height: {H:.2f}          px2deg: {px2deg:.5f}')

        LON, LAT = [], []
        for s in range(len(Is)):
            i, j = Is[s], Js[s]
            A,B,B_a = find_ABBa(i,j,mode, S, iss, CAMx, CAMy)

            Lon, Lat = LL_estimation(A, B, B_a, px2deg)
            LON.append(Lon)
            LAT.append(Lat)
        if len(LON)>3:
            LON_m = np.mean(filter_quartile(LON))
            LAT_m = np.mean(filter_quartile(LAT))
        else:
            LON_m = np.mean(LON)
            LAT_m = np.mean(LAT)

        print(f'LON: {LON_m:.3f}       LAT:{LAT_m:.3f}')
    else: print('Association was not possible...')

    dt = 10
    df = pd.read_csv(f'DATA/ephemeris sat/inclination zero/{dt} step size.csv', header=3, sep=';') 
    real_Latitudes, real_Longitudes, real_Altitudes = df['Lat (deg)'], df['Lon (deg)'], df['Alt (km)']
    real_Vxs,real_Vys,real_Vzs = df['x (km/sec)'], df['y (km/sec)'],df['z (km/sec)']

    real_X, real_Y, real_Z = [], [], []
    for i in range(len(df)):
        altitude = real_Altitudes[i]
        latitude = real_Latitudes[i]
        longitude = real_Longitudes[i]
        x, y, z = spherical2cartesian(altitude, latitude, longitude)
        real_X.append(x)
        real_Y.append(y)
        real_Z.append(z)
    real_X, real_Y, real_Z = np.array(real_X),np.array(real_Y),np.array(real_Z)


    Error = []
    if len(Js)!=0: 
        TRUE_POS = np.array([real_Longitudes[idx], real_Latitudes[idx]])
        # print(TRUE_POS)
        D = TRUE_POS - np.array([LON_m,LAT_m])
        ERROR_LL = D*deg2km
        Error.append(H-50)
        Error.append(ERROR_LL[0])
        Error.append(ERROR_LL[1])
        print(f'Error (km)| H: {Error[0]:.2f}Km    Lon:{Error[1]:.2f}Km    Lat:{Error[2]:.2f}Km')
    else: print('Association was not possible...')

    if len(Js)!=0: 
        Position_est = np.array([H, LAT_m, LON_m])
        Error_est = np.array([abs(Error[0]),abs(Error[1]),abs(Error[2])])
        return Position_est, Error_est
    else:
        return None 


In [10]:
FILE = pd.DataFrame(columns=['ID','H','LAT','LON','Error_H(km)','Error_LAT(km)','Error_LON(km)'])

for i in range(60):
    try:
        pos , err = find_pos(i)
    except TypeError: print('Sorry, cannot find position...')

    if (pos is not None) & (err is not None):
        new_row = {'ID':i, 'H':pos[0], 'LAT':pos[1],'LON':pos[2],'Error_H(km)':err[0],'Error_LAT(km)':err[1],'Error_LON(km)':err[2] }
        FILE = FILE.append(new_row, ignore_index=True)
        FILE.to_csv('Pos_Err_d100.csv')
print('Done!')

mputational time: 0.00 s
Mode:natural
Computational time: 0.07 s
Possible list Combinations: 9
Computational time: 0.03 s
Possible list Combinations: 1
Association was not possible...
Association was not possible...
Sorry, cannot find position...
Detection Time:0.46

 |████████████████████████████████████████████████████████████████████████████████████████████████████| 100.0% 


Total craters founded:7
Number of total combinations:210
Computational time: 0.00 s
 |████████████████████████████████████████████████████████████████████████████████████████████████████| 100.0% 
Total craters catalogued:6
Number of total combinations:60
Computational time: 0.00 s
Mode:natural
Computational time: 0.06 s
Possible list Combinations: 10
Computational time: 0.03 s
Possible list Combinations: 2
1 0
Height: 51.02          px2deg: 0.00396
LON: -129.806       LAT:0.015
Error (km)| H: 1.02Km    Lon:0.11Km    Lat:-0.44Km
Detection Time:0.43

 |█████████████████████████████████████████████████████████████

In [11]:
# H, Lon, Lat = [],[],[]
# for e in ERR:
#     H.append(e[0])
#     Lon.append(e[1])
#     Lat.append(e[2])
H, Lon, Lat, IDXs = FILE['Error_H(km)'], FILE['Error_LON(km)'], FILE['Error_LAT(km)'], FILE['ID']

In [12]:
plt.figure(dpi=150, figsize=(10,5), tight_layout=True)
plt.subplot(211)
plt.scatter(IDXs, H)
plt.xlabel('idx')
plt.ylabel('Km')
plt.title('Altitude Error')
plt.show()

plt.subplot(212)
plt.scatter(IDXs, H)
plt.xlabel('idx')
plt.ylabel('Km')
plt.ylim([0,2])
plt.title('Altitude Error')
plt.savefig('AltitudeError_d=150.jpg')
plt.show()

In [13]:
plt.figure(dpi=150, figsize=(10,5), tight_layout=True)
plt.subplot(211)
plt.scatter(IDXs, Lon)
plt.xlabel('idx')
plt.ylabel('Km')
plt.title('Longitude Error')
plt.show()

plt.subplot(212)
plt.scatter(IDXs, Lon)
plt.xlabel('idx')
plt.ylabel('Km')
plt.ylim([0,1.5])
plt.title('Longitude Error')
plt.savefig('LongitudeError_d=150.jpg')
plt.show()

In [14]:
plt.figure(dpi=150, figsize=(10,5), tight_layout=True)
plt.subplot(211)
plt.scatter(IDXs, Lat)
plt.xlabel('idx')
plt.ylabel('Km')
plt.title('Latitude Error')
plt.show()

plt.subplot(212)
plt.scatter(IDXs, Lat)
plt.xlabel('idx')
plt.ylabel('Km')
plt.ylim([0,1])
plt.title('Latitude Error')
plt.savefig('LatitudeError_d=150.jpg')
plt.show()